# Chapter 1: Mathematical Foundations and Implementation

## Building Math Libraries from Scratch

This notebook covers:
- Vector implementation (Vec2, Vec3, Vec4)
- Matrix implementation (Matrix3x3, Matrix4x4)
- Coordinate systems
- Transformation matrices
- Rotation representations (Euler angles, quaternions)

**Key References:** Marschner & Shirley Ch. 2, 5-6, Gambetta Ch. 1

---

## Setup and Imports

In [ ]:
import numpy as np
import math
from typing import Union, Tuple

# For visualization
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

print("Libraries imported successfully!")

---

## 1. Vector Mathematics - Theory

### 1.1 Vector Definition

A **vector** in 3D space is an ordered triple of real numbers:

$$\mathbf{v} = \begin{pmatrix} v_x \\ v_y \\ v_z \end{pmatrix} \in \mathbb{R}^3$$

Vectors represent both **direction** and **magnitude** (length).

### 1.2 Vector Operations

#### Addition and Subtraction
Vector addition follows the parallelogram rule:

$$\mathbf{u} + \mathbf{v} = \begin{pmatrix} u_x + v_x \\ u_y + v_y \\ u_z + v_z \end{pmatrix}$$

$$\mathbf{u} - \mathbf{v} = \begin{pmatrix} u_x - v_x \\ u_y - v_y \\ u_z - v_z \end{pmatrix}$$

#### Scalar Multiplication
Scaling a vector by scalar $k$:

$$k\mathbf{v} = \begin{pmatrix} kv_x \\ kv_y \\ kv_z \end{pmatrix}$$

### 1.3 Dot Product (Inner Product)

The **dot product** of two vectors is:

$$\mathbf{u} \cdot \mathbf{v} = u_x v_x + u_y v_y + u_z v_z = \|\mathbf{u}\| \|\mathbf{v}\| \cos\theta$$

where $\theta$ is the angle between the vectors.

**Properties:**
- $\mathbf{u} \cdot \mathbf{v} = \mathbf{v} \cdot \mathbf{u}$ (commutative)
- $\mathbf{u} \cdot \mathbf{u} = \|\mathbf{u}\|^2$
- $\mathbf{u} \cdot \mathbf{v} = 0 \iff \mathbf{u} \perp \mathbf{v}$ (orthogonal)

**Applications:** Lighting calculations (Lambertian shading), projections, angle computation.

### 1.4 Cross Product

The **cross product** produces a vector perpendicular to both input vectors:

$$\mathbf{u} \times \mathbf{v} = \begin{pmatrix} u_y v_z - u_z v_y \\ u_z v_x - u_x v_z \\ u_x v_y - u_y v_x \end{pmatrix}$$

**Magnitude:** $\|\mathbf{u} \times \mathbf{v}\| = \|\mathbf{u}\| \|\mathbf{v}\| \sin\theta$

**Properties:**
- $\mathbf{u} \times \mathbf{v} = -(\mathbf{v} \times \mathbf{u})$ (anti-commutative)
- $\mathbf{u} \times \mathbf{u} = \mathbf{0}$
- Right-hand rule determines direction
- $\mathbf{u} \times \mathbf{v} = \mathbf{0} \iff \mathbf{u} \parallel \mathbf{v}$ (parallel)

**Applications:** Surface normals, coordinate system construction, torque calculation.

### 1.5 Vector Magnitude and Normalization

**Magnitude (Length):**

$$\|\mathbf{v}\| = \sqrt{v_x^2 + v_y^2 + v_z^2} = \sqrt{\mathbf{v} \cdot \mathbf{v}}$$

**Unit Vector (Normalization):**

$$\hat{\mathbf{v}} = \frac{\mathbf{v}}{\|\mathbf{v}\|}, \quad \|\hat{\mathbf{v}}\| = 1$$

Normalized vectors represent pure direction without magnitude.

### 1.6 Reflection and Refraction

**Reflection** about normal $\mathbf{n}$:

$$\mathbf{r} = \mathbf{v} - 2(\mathbf{v} \cdot \mathbf{n})\mathbf{n}$$

**Refraction** (Snell's Law) with index ratio $\eta = \frac{n_1}{n_2}$:

$$\mathbf{t} = \eta \mathbf{v} + \left(\eta(\mathbf{v} \cdot \mathbf{n}) - \sqrt{1 - \eta^2(1 - (\mathbf{v} \cdot \mathbf{n})^2)}\right)\mathbf{n}$$

Total internal reflection occurs when discriminant $< 0$.

---

## 1. Vector Implementation

We'll build vector classes from scratch without using numpy for the core operations.

In [ ]:
class Vec3:
    """3D Vector implementation from scratch"""
    
    def __init__(self, x: float = 0.0, y: float = 0.0, z: float = 0.0):
        self.x = float(x)
        self.y = float(y)
        self.z = float(z)
    
    # Addition
    def __add__(self, other: 'Vec3') -> 'Vec3':
        return Vec3(self.x + other.x, self.y + other.y, self.z + other.z)
    
    # Subtraction
    def __sub__(self, other: 'Vec3') -> 'Vec3':
        return Vec3(self.x - other.x, self.y - other.y, self.z - other.z)
    
    # Scalar multiplication
    def __mul__(self, scalar: float) -> 'Vec3':
        return Vec3(self.x * scalar, self.y * scalar, self.z * scalar)
    
    def __rmul__(self, scalar: float) -> 'Vec3':
        return self.__mul__(scalar)
    
    # Scalar division
    def __truediv__(self, scalar: float) -> 'Vec3':
        return Vec3(self.x / scalar, self.y / scalar, self.z / scalar)
    
    # Negation
    def __neg__(self) -> 'Vec3':
        return Vec3(-self.x, -self.y, -self.z)
    
    # Dot product
    def dot(self, other: 'Vec3') -> float:
        """Compute dot product: v · u = vx*ux + vy*uy + vz*uz"""
        return self.x * other.x + self.y * other.y + self.z * other.z
    
    # Cross product
    def cross(self, other: 'Vec3') -> 'Vec3':
        """Compute cross product: v × u"""
        return Vec3(
            self.y * other.z - self.z * other.y,
            self.z * other.x - self.x * other.z,
            self.x * other.y - self.y * other.x
        )
    
    # Length (magnitude)
    def length(self) -> float:
        """Compute vector length: ||v|| = sqrt(x² + y² + z²)"""
        return math.sqrt(self.x**2 + self.y**2 + self.z**2)
    
    def length_squared(self) -> float:
        """Compute squared length (faster, no sqrt)"""
        return self.x**2 + self.y**2 + self.z**2
    
    # Normalization
    def normalize(self) -> 'Vec3':
        """Return normalized vector (unit length)"""
        length = self.length()
        if length > 0:
            return self / length
        return Vec3(0, 0, 0)
    
    def normalize_self(self) -> None:
        """Normalize this vector in-place"""
        length = self.length()
        if length > 0:
            self.x /= length
            self.y /= length
            self.z /= length
    
    # String representation
    def __repr__(self) -> str:
        return f"Vec3({self.x:.4f}, {self.y:.4f}, {self.z:.4f})"
    
    # Array-like access
    def __getitem__(self, index: int) -> float:
        if index == 0: return self.x
        elif index == 1: return self.y
        elif index == 2: return self.z
        else: raise IndexError("Vec3 index out of range")
    
    def to_list(self) -> list:
        return [self.x, self.y, self.z]

# Test Vec3
v1 = Vec3(1, 2, 3)
v2 = Vec3(4, 5, 6)

print("Vector Tests:")
print(f"v1 = {v1}")
print(f"v2 = {v2}")
print(f"v1 + v2 = {v1 + v2}")
print(f"v1 - v2 = {v1 - v2}")
print(f"v1 · v2 = {v1.dot(v2)}")
print(f"v1 × v2 = {v1.cross(v2)}")
print(f"||v1|| = {v1.length():.4f}")
print(f"normalize(v1) = {v1.normalize()}")

### Vector Utilities

### Gram-Schmidt Orthogonalization - Theory

The **Gram-Schmidt process** converts a set of linearly independent vectors into an **orthonormal basis**.

Given vectors $\mathbf{v}_1, \mathbf{v}_2, \mathbf{v}_3$, we compute orthonormal vectors $\mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3$:

**Step 1:** Normalize first vector
$$\mathbf{u}_1 = \frac{\mathbf{v}_1}{\|\mathbf{v}_1\|}$$

**Step 2:** Remove component of $\mathbf{v}_2$ along $\mathbf{u}_1$
$$\mathbf{u}_2' = \mathbf{v}_2 - (\mathbf{v}_2 \cdot \mathbf{u}_1)\mathbf{u}_1$$
$$\mathbf{u}_2 = \frac{\mathbf{u}_2'}{\|\mathbf{u}_2'\|}$$

**Step 3:** Remove components of $\mathbf{v}_3$ along $\mathbf{u}_1$ and $\mathbf{u}_2$
$$\mathbf{u}_3' = \mathbf{v}_3 - (\mathbf{v}_3 \cdot \mathbf{u}_1)\mathbf{u}_1 - (\mathbf{v}_3 \cdot \mathbf{u}_2)\mathbf{u}_2$$
$$\mathbf{u}_3 = \frac{\mathbf{u}_3'}{\|\mathbf{u}_3'\|}$$

**Result:** $\mathbf{u}_1, \mathbf{u}_2, \mathbf{u}_3$ form an orthonormal basis:
- $\mathbf{u}_i \cdot \mathbf{u}_j = \delta_{ij} = \begin{cases} 1 & i=j \\ 0 & i \neq j \end{cases}$
- $\|\mathbf{u}_i\| = 1$ for all $i$

**Applications:** Constructing camera coordinate systems, QR decomposition.

### Gram-Schmidt Orthogonalization

---

## 2. Matrix Mathematics - Theory

### 2.1 Matrix Definition

A **matrix** is a rectangular array of numbers. For 3D graphics, we primarily use $4 \times 4$ matrices:

$$\mathbf{M} = \begin{pmatrix} 
m_{00} & m_{01} & m_{02} & m_{03} \\
m_{10} & m_{11} & m_{12} & m_{13} \\
m_{20} & m_{21} & m_{22} & m_{23} \\
m_{30} & m_{31} & m_{32} & m_{33}
\end{pmatrix}$$

### 2.2 Homogeneous Coordinates

We use **homogeneous coordinates** to represent 3D points and vectors uniformly:

**Point:** $\mathbf{p} = \begin{pmatrix} x \\ y \\ z \\ 1 \end{pmatrix}$ (affected by translation)

**Vector:** $\mathbf{v} = \begin{pmatrix} x \\ y \\ z \\ 0 \end{pmatrix}$ (not affected by translation)

Converting from homogeneous to Cartesian: $\begin{pmatrix} x \\ y \\ z \\ w \end{pmatrix} \rightarrow \begin{pmatrix} x/w \\ y/w \\ z/w \end{pmatrix}$

### 2.3 Matrix Operations

#### Matrix-Matrix Multiplication

$$(\mathbf{AB})_{ij} = \sum_{k=0}^{3} a_{ik} b_{kj}$$

**Properties:**
- **Not commutative:** $\mathbf{AB} \neq \mathbf{BA}$ (generally)
- **Associative:** $(\mathbf{AB})\mathbf{C} = \mathbf{A}(\mathbf{BC})$
- **Identity:** $\mathbf{MI} = \mathbf{IM} = \mathbf{M}$

#### Matrix-Vector Multiplication

$$\mathbf{Mv} = \begin{pmatrix} 
m_{00}v_x + m_{01}v_y + m_{02}v_z + m_{03}v_w \\
m_{10}v_x + m_{11}v_y + m_{12}v_z + m_{13}v_w \\
m_{20}v_x + m_{21}v_y + m_{22}v_z + m_{23}v_w \\
m_{30}v_x + m_{31}v_y + m_{32}v_z + m_{33}v_w
\end{pmatrix}$$

### 2.4 Matrix Transpose

Swap rows and columns:

$$\mathbf{M}^T_{ij} = \mathbf{M}_{ji}$$

**Properties:**
- $(\mathbf{M}^T)^T = \mathbf{M}$
- $(\mathbf{AB})^T = \mathbf{B}^T\mathbf{A}^T$

### 2.5 Matrix Determinant

For a $4 \times 4$ matrix, compute via cofactor expansion:

$$\det(\mathbf{M}) = \sum_{j=0}^{3} (-1)^j m_{0j} \det(\mathbf{M}_{0j})$$

where $\mathbf{M}_{ij}$ is the $3 \times 3$ minor obtained by removing row $i$ and column $j$.

**Properties:**
- $\det(\mathbf{AB}) = \det(\mathbf{A})\det(\mathbf{B})$
- $\det(\mathbf{M}^T) = \det(\mathbf{M})$
- $\det(\mathbf{M}) = 0 \iff \mathbf{M}$ is singular (non-invertible)

### 2.6 Identity Matrix

$$\mathbf{I} = \begin{pmatrix} 
1 & 0 & 0 & 0 \\
0 & 1 & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}$$

The identity matrix leaves vectors unchanged: $\mathbf{Iv} = \mathbf{v}$

---

## 2. Matrix Implementation

---

## 3. Transformation Matrices - Theory

Transformation matrices allow us to represent **geometric transformations** (translation, rotation, scale) as matrix operations.

### 3.1 Translation

Move a point by offset $\mathbf{t} = (t_x, t_y, t_z)$:

$$\mathbf{T}(t_x, t_y, t_z) = \begin{pmatrix} 
1 & 0 & 0 & t_x \\
0 & 1 & 0 & t_y \\
0 & 0 & 1 & t_z \\
0 & 0 & 0 & 1
\end{pmatrix}$$

Applying to point: $\mathbf{p}' = \mathbf{T}\mathbf{p} = \mathbf{p} + \mathbf{t}$

### 3.2 Scale

Scale along each axis by factors $(s_x, s_y, s_z)$:

$$\mathbf{S}(s_x, s_y, s_z) = \begin{pmatrix} 
s_x & 0 & 0 & 0 \\
0 & s_y & 0 & 0 \\
0 & 0 & s_z & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}$$

Uniform scaling: $s_x = s_y = s_z = s$

### 3.3 Rotation Matrices

#### Rotation around X-axis by angle $\theta$:

$$\mathbf{R}_x(\theta) = \begin{pmatrix} 
1 & 0 & 0 & 0 \\
0 & \cos\theta & -\sin\theta & 0 \\
0 & \sin\theta & \cos\theta & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}$$

#### Rotation around Y-axis:

$$\mathbf{R}_y(\theta) = \begin{pmatrix} 
\cos\theta & 0 & \sin\theta & 0 \\
0 & 1 & 0 & 0 \\
-\sin\theta & 0 & \cos\theta & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}$$

#### Rotation around Z-axis:

$$\mathbf{R}_z(\theta) = \begin{pmatrix} 
\cos\theta & -\sin\theta & 0 & 0 \\
\sin\theta & \cos\theta & 0 & 0 \\
0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}$$

### 3.4 Rodrigues' Rotation Formula

Rotation by angle $\theta$ around arbitrary axis $\mathbf{k} = (k_x, k_y, k_z)$ (unit vector):

$$\mathbf{R}(\mathbf{k}, \theta) = \mathbf{I} + (\sin\theta)\mathbf{K} + (1-\cos\theta)\mathbf{K}^2$$

where $\mathbf{K}$ is the skew-symmetric cross-product matrix:

$$\mathbf{K} = \begin{pmatrix} 
0 & -k_z & k_y \\
k_z & 0 & -k_x \\
-k_y & k_x & 0
\end{pmatrix}$$

**Explicit form** (using $c = \cos\theta$, $s = \sin\theta$, $t = 1-\cos\theta$):

$$\mathbf{R}(\mathbf{k}, \theta) = \begin{pmatrix} 
tk_x^2 + c & tk_xk_y - sk_z & tk_xk_z + sk_y & 0 \\
tk_xk_y + sk_z & tk_y^2 + c & tk_yk_z - sk_x & 0 \\
tk_xk_z - sk_y & tk_yk_z + sk_x & tk_z^2 + c & 0 \\
0 & 0 & 0 & 1
\end{pmatrix}$$

### 3.5 Properties of Rotation Matrices

- **Orthogonal:** $\mathbf{R}^T\mathbf{R} = \mathbf{I}$, so $\mathbf{R}^{-1} = \mathbf{R}^T$
- **Determinant:** $\det(\mathbf{R}) = 1$ (preserves orientation)
- **Length-preserving:** $\|\mathbf{Rv}\| = \|\mathbf{v}\|$ (isometry)
- **Angle-preserving:** $(\mathbf{Ru}) \cdot (\mathbf{Rv}) = \mathbf{u} \cdot \mathbf{v}$

### 3.6 Transformation Composition

Transformations combine via matrix multiplication (right to left):

$$\mathbf{M} = \mathbf{T} \mathbf{R} \mathbf{S}$$

Applied to point: $\mathbf{p}' = \mathbf{TRS}\mathbf{p}$

**Order matters!** $\mathbf{TR} \neq \mathbf{RT}$ (translation then rotation ≠ rotation then translation)

---

## 3. Transformation Matrices - Implementation

In [ ]:
---

## 4. Quaternions - Theory

### 4.1 Quaternion Definition

A **quaternion** is an extension of complex numbers to 4D:

$$\mathbf{q} = w + x\mathbf{i} + y\mathbf{j} + z\mathbf{k} = (w, x, y, z) = (w, \mathbf{v})$$

where $\mathbf{i}^2 = \mathbf{j}^2 = \mathbf{k}^2 = \mathbf{ijk} = -1$

- $w$ is the **scalar part** (real component)
- $(x, y, z) = \mathbf{v}$ is the **vector part** (imaginary components)

### 4.2 Quaternion Operations

#### Multiplication (Hamilton Product)

$$\mathbf{q}_1 \mathbf{q}_2 = (w_1w_2 - \mathbf{v}_1 \cdot \mathbf{v}_2,\, w_1\mathbf{v}_2 + w_2\mathbf{v}_1 + \mathbf{v}_1 \times \mathbf{v}_2)$$

Expanded form:
$$\mathbf{q}_1 \mathbf{q}_2 = \begin{pmatrix}
w_1w_2 - x_1x_2 - y_1y_2 - z_1z_2 \\
w_1x_2 + x_1w_2 + y_1z_2 - z_1y_2 \\
w_1y_2 - x_1z_2 + y_1w_2 + z_1x_2 \\
w_1z_2 + x_1y_2 - y_1x_2 + z_1w_2
\end{pmatrix}$$

**Not commutative:** $\mathbf{q}_1\mathbf{q}_2 \neq \mathbf{q}_2\mathbf{q}_1$

#### Conjugate

$$\mathbf{q}^* = (w, -\mathbf{v}) = (w, -x, -y, -z)$$

#### Magnitude (Norm)

$$\|\mathbf{q}\| = \sqrt{w^2 + x^2 + y^2 + z^2}$$

#### Normalization

$$\hat{\mathbf{q}} = \frac{\mathbf{q}}{\|\mathbf{q}\|}$$

**Unit quaternion:** $\|\mathbf{q}\| = 1 \implies \mathbf{q}\mathbf{q}^* = 1$

### 4.3 Quaternion Rotations

A **unit quaternion** can represent a rotation:

$$\mathbf{q} = \cos\frac{\theta}{2} + \sin\frac{\theta}{2}(k_x\mathbf{i} + k_y\mathbf{j} + k_z\mathbf{k})$$

where $\mathbf{k} = (k_x, k_y, k_z)$ is the rotation axis (unit vector) and $\theta$ is the rotation angle.

**Compact form:**
$$\mathbf{q} = \left(\cos\frac{\theta}{2}, \sin\frac{\theta}{2}\mathbf{k}\right)$$

#### Rotating a Vector

To rotate vector $\mathbf{v}$ by quaternion $\mathbf{q}$:

$$\mathbf{v}' = \mathbf{q} \mathbf{v} \mathbf{q}^*$$

where $\mathbf{v}$ is treated as quaternion $(0, \mathbf{v})$.

### 4.4 Euler Angles to Quaternion

Given Euler angles (roll $\phi$, pitch $\theta$, yaw $\psi$):

$$\mathbf{q} = \begin{pmatrix}
\cos\frac{\psi}{2}\cos\frac{\theta}{2}\cos\frac{\phi}{2} + \sin\frac{\psi}{2}\sin\frac{\theta}{2}\sin\frac{\phi}{2} \\
\sin\frac{\psi}{2}\cos\frac{\theta}{2}\cos\frac{\phi}{2} - \cos\frac{\psi}{2}\sin\frac{\theta}{2}\sin\frac{\phi}{2} \\
\cos\frac{\psi}{2}\sin\frac{\theta}{2}\cos\frac{\phi}{2} + \sin\frac{\psi}{2}\cos\frac{\theta}{2}\sin\frac{\phi}{2} \\
\cos\frac{\psi}{2}\cos\frac{\theta}{2}\sin\frac{\phi}{2} - \sin\frac{\psi}{2}\sin\frac{\theta}{2}\cos\frac{\phi}{2}
\end{pmatrix}$$

### 4.5 Spherical Linear Interpolation (SLERP)

**SLERP** interpolates between two quaternions along the shortest arc on a 4D hypersphere:

$$\text{slerp}(\mathbf{q}_1, \mathbf{q}_2, t) = \frac{\sin((1-t)\Omega)}{\sin\Omega}\mathbf{q}_1 + \frac{\sin(t\Omega)}{\sin\Omega}\mathbf{q}_2$$

where $\Omega = \arccos(\mathbf{q}_1 \cdot \mathbf{q}_2)$ is the angle between quaternions.

**For close quaternions** ($\mathbf{q}_1 \cdot \mathbf{q}_2 > 0.9995$), use linear interpolation:
$$\text{lerp}(\mathbf{q}_1, \mathbf{q}_2, t) = (1-t)\mathbf{q}_1 + t\mathbf{q}_2$$
followed by normalization.

### 4.6 Advantages of Quaternions

✅ **Compact:** 4 numbers vs 9 for rotation matrix  
✅ **No gimbal lock:** Unlike Euler angles  
✅ **Efficient interpolation:** SLERP produces smooth rotations  
✅ **Numerically stable:** Easy to renormalize  
✅ **Efficient composition:** Faster than matrix multiplication  

**Disadvantages:**
- Not intuitive (harder to visualize than Euler angles)
- Require conversion to matrices for rendering

---

## 4. Quaternions - Implementation

---

## 3. Transformation Matrices

In [ ]:
def translation_matrix(tx: float, ty: float, tz: float) -> Mat4:
    """Create translation matrix"""
    m = Mat4.identity()
    m[0][3] = tx
    m[1][3] = ty
    m[2][3] = tz
    return m

def scale_matrix(sx: float, sy: float, sz: float) -> Mat4:
    """Create scale matrix"""
    m = Mat4.zeros()
    m[0][0] = sx
    m[1][1] = sy
    m[2][2] = sz
    m[3][3] = 1.0
    return m

def rotation_x(angle: float) -> Mat4:
    """Rotation around X-axis (angle in radians)"""
    c = math.cos(angle)
    s = math.sin(angle)
    m = Mat4.identity()
    m[1][1] = c
    m[1][2] = -s
    m[2][1] = s
    m[2][2] = c
    return m

def rotation_y(angle: float) -> Mat4:
    """Rotation around Y-axis (angle in radians)"""
    c = math.cos(angle)
    s = math.sin(angle)
    m = Mat4.identity()
    m[0][0] = c
    m[0][2] = s
    m[2][0] = -s
    m[2][2] = c
    return m

def rotation_z(angle: float) -> Mat4:
    """Rotation around Z-axis (angle in radians)"""
    c = math.cos(angle)
    s = math.sin(angle)
    m = Mat4.identity()
    m[0][0] = c
    m[0][1] = -s
    m[1][0] = s
    m[1][1] = c
    return m

def rotation_axis_angle(axis: Vec3, angle: float) -> Mat4:
    """Rotation around arbitrary axis using Rodrigues' formula"""
    # Normalize axis
    axis = axis.normalize()
    c = math.cos(angle)
    s = math.sin(angle)
    t = 1 - c
    
    x, y, z = axis.x, axis.y, axis.z
    
    m = Mat4([
        [t*x*x + c,   t*x*y - s*z, t*x*z + s*y, 0],
        [t*x*y + s*z, t*y*y + c,   t*y*z - s*x, 0],
        [t*x*z - s*y, t*y*z + s*x, t*z*z + c,   0],
        [0,           0,           0,           1]
    ])
    return m

# Test transformations
print("Translation by (1, 2, 3):")
print(translation_matrix(1, 2, 3))

print("\nRotation around Z-axis by 90°:")
print(rotation_z(math.pi/2))

# Test composition
v = Vec3(1, 0, 0)
rot = rotation_z(math.pi/2)
v_rotated = rot.mul_vec3(v)
print(f"\nRotate (1,0,0) by 90° around Z: {v_rotated}")

---

## 6. Practical Examples

Let's see how these mathematical primitives work together in practical scenarios.

In [ ]:
# Example 1: Building a Camera Coordinate System
# Given camera position and target, construct orthonormal basis

def build_camera_basis(eye: Vec3, target: Vec3, world_up: Vec3) -> Tuple[Vec3, Vec3, Vec3]:
    """Build camera coordinate system using Gram-Schmidt"""
    # Forward direction (from eye to target)
    forward = (target - eye).normalize()
    
    # Right direction (perpendicular to forward and up)
    right = forward.cross(world_up).normalize()
    
    # Up direction (perpendicular to both)
    up = right.cross(forward).normalize()
    
    return right, up, forward

# Test camera system
eye = Vec3(0, 0, 5)
target = Vec3(0, 0, 0)
world_up = Vec3(0, 1, 0)

right, up, forward = build_camera_basis(eye, target, world_up)
print("Example 1: Camera Coordinate System")
print(f"Right:   {right}")
print(f"Up:      {up}")
print(f"Forward: {forward}")
print(f"Orthogonal check - right·up = {right.dot(up):.6f}")
print()

# Example 2: Transforming objects in 3D
print("Example 2: Object Transformation Pipeline")

# Create a point
point = Vec3(1, 0, 0)
print(f"Original point: {point}")

# Apply transformations: Scale -> Rotate -> Translate
scale = scale_matrix(2, 2, 2)
rotate = rotation_z(math.pi / 4)  # 45 degrees
translate = translation_matrix(1, 2, 3)

# Compose transformations (right to left)
transform = translate @ rotate @ scale

# Apply to point
transformed = transform.mul_vec3(point)
print(f"After Scale(2) -> Rotate(45°) -> Translate(1,2,3): {transformed}")
print()

# Example 3: Quaternion interpolation for smooth rotation
print("Example 3: Smooth Rotation with SLERP")

# Start: no rotation
q_start = Quaternion.from_axis_angle(Vec3(0, 1, 0), 0)

# End: 180 degree rotation around Y-axis
q_end = Quaternion.from_axis_angle(Vec3(0, 1, 0), math.pi)

# Point to rotate
p = Vec3(1, 0, 0)

print("Interpolating rotation from 0° to 180° around Y-axis:")
for t in [0.0, 0.25, 0.5, 0.75, 1.0]:
    q_interpolated = slerp(q_start, q_end, t)
    p_rotated = q_interpolated.rotate_vector(p)
    angle_deg = 180 * t
    print(f"  t={t:.2f} ({angle_deg:5.1f}°): {p_rotated}")
print()

# Example 4: Reflection and Refraction
print("Example 4: Light Ray Reflection and Refraction")

# Incident ray (pointing down and right)
incident = Vec3(1, -1, 0).normalize()
normal = Vec3(0, 1, 0)  # Surface normal (upward)

# Reflection
reflected = reflect(incident, normal)
print(f"Incident ray:  {incident}")
print(f"Surface normal: {normal}")
print(f"Reflected ray:  {reflected}")

# Refraction (air to glass, eta ≈ 0.67)
eta = 1.0 / 1.5  # n_air / n_glass
refracted = refract(incident, normal, eta)
print(f"Refracted ray (air→glass): {refracted}")
print()

# Example 5: Matrix-Quaternion equivalence
print("Example 5: Rotation Matrix vs Quaternion")

axis = Vec3(1, 1, 1).normalize()
angle = math.pi / 3  # 60 degrees

# Using rotation matrix
mat_rot = rotation_axis_angle(axis, angle)
v_mat = mat_rot.mul_vec3(Vec3(1, 0, 0))

# Using quaternion
quat_rot = Quaternion.from_axis_angle(axis, angle)
v_quat = quat_rot.rotate_vector(Vec3(1, 0, 0))

print(f"Rotating (1,0,0) by 60° around {axis}:")
print(f"  Matrix result:     {v_mat}")
print(f"  Quaternion result: {v_quat}")
print(f"  Difference: {(v_mat - v_quat).length():.10f}")
print()

# Example 6: Composing rotations with quaternions
print("Example 6: Composing Multiple Rotations")

# Rotate 90° around Z, then 90° around X
q1 = Quaternion.from_axis_angle(Vec3(0, 0, 1), math.pi/2)  # Z rotation
q2 = Quaternion.from_axis_angle(Vec3(1, 0, 0), math.pi/2)  # X rotation

# Compose: apply q1 first, then q2
q_combined = q2 * q1

point = Vec3(1, 0, 0)
result1 = q1.rotate_vector(point)
result2 = q2.rotate_vector(result1)
result_combined = q_combined.rotate_vector(point)

print(f"Original point: {point}")
print(f"After Z-rotation: {result1}")
print(f"After X-rotation: {result2}")
print(f"Using combined quaternion: {result_combined}")
print(f"Difference: {(result2 - result_combined).length():.10f}")

---

## 5. Visualization

In [ ]:
def plot_vectors_3d(vectors: list, labels: list = None, colors: list = None):
    """Plot 3D vectors"""
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    if labels is None:
        labels = [f"v{i}" for i in range(len(vectors))]
    if colors is None:
        colors = plt.cm.rainbow(np.linspace(0, 1, len(vectors)))
    
    for v, label, color in zip(vectors, labels, colors):
        ax.quiver(0, 0, 0, v.x, v.y, v.z, 
                 color=color, arrow_length_ratio=0.1, linewidth=2, label=label)
    
    # Set limits
    max_val = max(max(abs(v.x), abs(v.y), abs(v.z)) for v in vectors) * 1.2
    ax.set_xlim([-max_val, max_val])
    ax.set_ylim([-max_val, max_val])
    ax.set_zlim([-max_val, max_val])
    
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    plt.title('3D Vector Visualization')
    plt.show()

# Visualize some vectors
v1 = Vec3(1, 0, 0)
v2 = Vec3(0, 1, 0)
v3 = Vec3(0, 0, 1)
v_cross = v1.cross(v2)

plot_vectors_3d(
    [v1, v2, v3, v_cross],
    ['X-axis', 'Y-axis', 'Z-axis', 'X × Y'],
    ['red', 'green', 'blue', 'purple']
)

---

## 6. Exercises

Try implementing these on your own:

1. **Vec2 and Vec4**: Create Vec2 and Vec4 classes similar to Vec3

2. **Matrix Inverse**: Implement matrix inversion for Mat4 using Gauss-Jordan elimination

3. **LookAt Matrix**: Implement a camera "look-at" transformation matrix:
   ```python
   def look_at(eye: Vec3, target: Vec3, up: Vec3) -> Mat4:
       # Create view matrix
       pass
   ```

4. **Perspective Projection**: Create a perspective projection matrix:
   ```python
   def perspective(fov: float, aspect: float, near: float, far: float) -> Mat4:
       # Create perspective projection
       pass
   ```

5. **Euler to Quaternion**: Create a function to convert Euler angles to quaternion (already partially done)

6. **Matrix Decomposition**: Decompose a transformation matrix into translation, rotation, and scale components

In [ ]:
# Your exercise implementations here


---

## Summary

In this chapter, you learned to implement:

✅ **Vec3** - Complete 3D vector class with all operations  
✅ **Mat4** - 4×4 matrix for transformations  
✅ **Transformations** - Translation, rotation, scale matrices  
✅ **Quaternions** - Rotation representation and SLERP  
✅ **Gram-Schmidt** - Orthonormalization algorithm  

These math primitives form the foundation for all 3D graphics operations!

**Next Chapter:** Framebuffer and 2D Rasterization